<a href="https://colab.research.google.com/github/omar-conti/public/blob/main/FASTWEB_Unpack_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--- 
# Unpac FASTWEB
---

1.   Incollare ID file Dettaglio e ID cartella
2.   Run all (CTRL + F9)
3.   Dare autorizzazione
   






# Autorizzazione

In [ ]:
!pip install --upgrade gspread

from google.colab import auth
import pandas as pd
import gspread
from oauth2client.client import GoogleCredentials


# ID

In [ ]:
#@title ID File e Cartella
variable_name = 12
id_google_sheet = "1P-zdB70f-cUieks6FPQu2tLWlfoGN_u6zcl6uce89E0" #@param {type:"string"}
nome_sheet_D = 'DB_PAYOUT_I'
nome_sheet_R = 'R'
nome_sheet_G = 'GARE'
id_folder = "1CTIbS2Wj66ffoYT1dwNFr2Gzn2WUxNhY" #@param {type:"string"}

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_key(id_google_sheet)
ws_D = wb.worksheet(nome_sheet_D)
ws_R = wb.worksheet(nome_sheet_R)
ws_G = wb.worksheet(nome_sheet_G)


# CODICE

In [ ]:
ws_config = wb.worksheet('Config_Divisione')

rows_config = ws_config.get('E3:G5')

nome_file = ws_config.get('B3:B4')
df1 = pd.DataFrame.from_records(rows_config[1:],columns=rows_config[0])
ss=  [df1.columns.tolist()] + df1.to_numpy().tolist()

l1 = []
for s in ss:
  col = s[2].split(',')
  n1 = s[0]
  n2 = s[1]
  l1.append(n1)
  l1.append(n2)
  l1.append(col)

lista_r = l1[2]
lista_i = l1[5]
lista_gare = l1[8]
post = nome_file[0]
pre = nome_file[1]

In [ ]:
# get_all_values gives a list of rows.
rows_R = ws_R.get_all_values()
df_R = pd.DataFrame.from_records(rows_R[1:],columns=rows_R[0])

rows_D = ws_D.get_all_values()
df_D = pd.DataFrame.from_records(rows_D[1:],columns=rows_D[0])

rows_G = ws_G.get_all_values()
df_G = pd.DataFrame.from_records(rows_G[1:],columns=rows_G[0])

salesRepSeries = df_R
UniqueSalesPeople = salesRepSeries.drop_duplicates(subset = ['Partner Tool', 'FlagConsorzio'],  keep = 'last').reset_index(drop = True)
UniqueSalesPeople = UniqueSalesPeople[['Partner Tool', 'FlagConsorzio']]
UniqueSalesPeople= UniqueSalesPeople[(UniqueSalesPeople['Partner Tool']!="")]
UniquePartner= UniqueSalesPeople.to_numpy().tolist()


In [ ]:
def autoResize (spreadsheetName,sheetName):
  ss = gc.open(spreadsheetName)
  sheetId = ss.worksheet(sheetName)._properties['sheetId']
  body = {
      "requests": [
          {
              "autoResizeDimensions": {
                  "dimensions": {
                      "sheetId": sheetId,
                      "dimension": "COLUMNS",
                      "startIndex": 0,  # Please set the column index.
                      "endIndex": 60  # Please set the column index.
                  }
              }
          }
      ]
  }
  res = ss.batch_update(body)

In [ ]:
def createSpreadsheet(salesRep,flag):
  riepilogo = df_R[df_R["Partner Tool"] == salesRep]
  dettaglio = df_D[df_D["PARTNER"] == salesRep]
  gare = df_G[df_G["PARTNER"] == salesRep]
  
 
  lista_col = lista_i


  riepilogo1 = riepilogo[lista_r]
  r_list = [riepilogo1.columns.tolist()] + riepilogo1.to_numpy().tolist()


  dettaglio = dettaglio[lista_col]
  d_list = [dettaglio.columns.tolist()] + dettaglio.to_numpy().tolist()
  
  gare1 = gare[lista_gare]
  g_list = [gare.columns.tolist()] + gare.to_numpy().tolist()



  titolo = pre[0] + '_' + salesRep + '_' + post[0]
  ss = gc.create(titolo,id_folder)

  nws = ss.sheet1
  nws2 = ss.add_worksheet(title='Dettaglio', rows="100", cols="20")
  nws3 = ss.add_worksheet(title='Gare', rows="50", cols="15")

  nws.update_title('R')

  nws.update("A1",r_list,value_input_option="USER_ENTERED")
  nws2.update("A1",d_list,value_input_option="USER_ENTERED")
  nws3.update("A1",g_list,value_input_option="USER_ENTERED")

  nws.format("A1:J1", {
    "backgroundColor": {
        "red": 0.0,
        "green": 0.0,
        "blue": 140.0
    },
    "horizontalAlignment": "LEFT",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 10,
      "bold": True
    }
  })

  nws2.format("A1:1", {
    "backgroundColor": {
        "red": 0.0,
        "green": 0.0,
        "blue": 140.0
    },
    "horizontalAlignment": "LEFT",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 10,
      "bold": True
    }
  })

  nws3.format("A1:1", {
    "backgroundColor": {
        "red": 0.0,
        "green": 0.0,
        "blue": 140.0
    },
    "horizontalAlignment": "LEFT",
    "textFormat": {
      "foregroundColor": {
        "red": 1.0,
        "green": 1.0,
        "blue": 1.0
      },
      "fontSize": 10,
      "bold": True
    }
  })

  autoResize(titolo,'R')
  autoResize(titolo,'Dettaglio')
  autoResize(titolo,'Gare')

for p in UniquePartner:
  createSpreadsheet(p[0],p[1])